In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import sys                                                            #
sys.path.append('/content/drive/My Drive/embedding_method')                      #
!pip install wikipedia                                                #   
!pip install unidecode                                                #         Importing necessary modules
import numpy as np                                                    #
from keras.models import Sequential                                   #
from keras.layers import Dense                                        #
from keras.layers import Embedding                                        
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
from config_2 import language_tags, max_letters
import random
from itertools import chain
from functions_2 import generate_dictionary , process, convert_dic_to_vector
import functions_2
import numpy as np
from itertools import chain 
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import GlobalAveragePooling1D


In [5]:
from functions_2 import convert_dic_to_vector
word_data = []
language_data = []
master_dic = []

count = 0

for tag in language_tags.keys():                                                #
    print('generating dictionary for ' + tag)                                   #   Generating dictionary
    dic = functions_2.generate_dictionary(tag, max_letters)                     #
    master_dic.append(dic)                                                      #     
    
# print(master_dic) 
    
    vct = functions_2.convert_dic_to_vector(dic, max_letters)                   #                   
    for vector in vct:                                                          #
        word_data.append(vector)                                                #
    output_vct = functions_2.create_output_vector(count, len(language_tags))    #           Labels processing
    for i in range(len(vct)):                                                   #
        language_data.append(output_vct)                                        #
    count += 1                                                                  #


flatten_list = list(chain.from_iterable(master_dic))


tokenizer = Tokenizer(oov_token="<NewLang>")                                    #
tokenizer.fit_on_texts(flatten_list)                                            #    Tokenizing the processed words obtained
alpha_1=tokenizer.texts_to_sequences(flatten_list)                              #


arr = []                                                                        #
for i in range(len(word_data)):                                                 #
    entry = []                                                                  #
    entry.append(alpha_1[i])                                                    #     Organizing the data together
    for digit in language_data[i]:                                              #      
        entry.append(float(digit))                                              #
    arr.append(entry)                                                           #


generating dictionary for en
generating dictionary for de
generating dictionary for fr


In [0]:
inputs=[]
labels=[]
for i in range (len(arr)):                                                      #
  inputs.append(arr[i][0])                                                      #    Seperating inputs and labels
  labels.append(arr[i][1:])                                                     # 

inputs=np.asarray(inputs)                                                                           #     Converting to numpy array compatible format
labels=np.asarray(labels)                                                                           #

x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)                 #   Splitting validation and test set

In [7]:
word_index=tokenizer.word_index
print(word_index)


network = Sequential()                                                                  
network.add(Embedding(input_dim=9710,output_dim=8))                                     #
network.add(GlobalAveragePooling1D())                                                   #
network.add(Dense(6, activation='relu'))                                                #                         Defining model and
network.add(Dense(3, activation='softmax'))                                             #                         its parameters

network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])     #
network.summary()                                                                       #

{'<NewLang>': 1, 'the': 2, 'de': 3, 'in': 4, 'of': 5, 'la': 6, 'der': 7, 'die': 8, 'and': 9, 'a': 10, 'und': 11, 'london': 12, 'des': 13, 'et': 14, 'les': 15, 'le': 16, 'figures': 17, 'to': 18, 'is': 19, 'gott': 20, 'als': 21, 'dans': 22, 'en': 23, 'den': 24, 'du': 25, 'style': 26, 'gotter': 27, 'city': 28, 'par': 29, 'im': 30, 'une': 31, 'un': 32, 'von': 33, 'est': 34, 'qui': 35, 'as': 36, 'are': 37, 'by': 38, 'zu': 39, 'que': 40, 'was': 41, 'mit': 42, 'ou': 43, 'sur': 44, 'ist': 45, 'das': 46, 'eine': 47, 'for': 48, 'oder': 49, 'dem': 50, 'comme': 51, 'sich': 52, 'rhetorique': 53, 'auch': 54, 'with': 55, 'has': 56, 'wird': 57, 'pour': 58, 'dass': 59, 'il': 60, 'on': 61, 'an': 62, 'plus': 63, 'au': 64, 'from': 65, 'werden': 66, 'wie': 67, 'isbn': 68, 'nicht': 69, 'sont': 70, 'figure': 71, 'per': 72, 'fur': 73, 'wurde': 74, 'at': 75, 'it': 76, 'ein': 77, 'bei': 78, 'aus': 79, 'mots': 80, 'sind': 81, 'sens': 82, 'most': 83, 'paris': 84, 'londons': 85, 'world': 86, 'cent': 87, 'gottes': 

In [8]:
filepath = "weights.hdf5"                                                                                     #
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')         #
tboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)                                   #   Defining checkpoints and other parameters
callbacks_list = [checkpoint, tboard]                                                                         #
                                                                                                              #   good that our model is obtaining 92.5% accuracy on validation set
                                                                                                              #
network.fit(x_train, y_train, epochs=100, batch_size=1000, validation_data=(x_test, y_test), callbacks=callbacks_list)     # model training   




Train on 32471 samples, validate on 5731 samples







Epoch 1/100
32471/32471 [==============================] - 1s 31us/step - loss: 0.6325 - acc: 0.6667 - val_loss: 0.6267 - val_acc: 0.6667

Epoch 00001: val_acc improved from -inf to 0.66667, saving model to weights.hdf5

Epoch 2/100
32471/32471 [==============================] - 0s 5us/step - loss: 0.6174 - acc: 0.6667 - val_loss: 0.6065 - val_acc: 0.6667

Epoch 00002: val_acc did not improve from 0.66667
Epoch 3/100
32471/32471 [==============================] - 0s 4us/step - loss: 0.5879 - acc: 0.6667 - val_loss: 0.5718 - val_acc: 0.6704

Epoch 00003: val_acc improved from 0.66667 to 0.67039, saving model to weights.hdf5
Epoch 4/100
32471/32471 [==============================] - 0s 5us/step - loss: 0.5419 - acc: 0.6988 - val_loss: 0.5238 - val_acc: 0.7328

Epoch 00004: val_acc improved from 0.67039 to 0.73280, saving model to weights.hdf5
Epoch 5/100
32471/32471 [==============================] - 0s 4us/step - loss: 0.4832 - 

In [9]:
model = Sequential()                                                                    #
model.add(Embedding(input_dim=9710,output_dim=8))                                       #
model.add(GlobalAveragePooling1D())                                                     #
model.add(Dense(6,activation='relu'))                                                   #     Loading weights
model.add(Dense(3, activation='softmax'))                                               #       
model.load_weights('weights.hdf5')                                                      #
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])       #




file4 = open('/content/drive/My Drive/embedding_method/test.txt', 'r')                               #   Opening files
Lines = file4.readlines()                                                               #
punctuations = ".!?,;'/\n"                                                              #   One can add on additional punctuations here 
no_punct = ""                                                                           #
alpha=[]                                                                                #

for line in Lines:                                                                      #
    temp=line.lower()                                                                   #
    for char in temp:                                                                   #   Processing 
        if char in punctuations:                                                        #   text.txt file 
            temp=temp.replace(char,"")                                                  #   into an array
    alpha.append(temp)                                                                  #
                                                                                        #
alpha1=[]                                                                               #
for j in range (len(alpha)):                                                            #
  alpha1.append(process(alpha[j],200))                                                  #


alpha1 = list(chain.from_iterable(alpha1))                                              #     Squashing 2-d list into 1-d

print(alpha1)                                                                           #     Printing the words extracted

dictio=[]                                                                               #

zer=np.zeros((1,3))                                                                     #


tc=tokenizer.texts_to_sequences(alpha1)                                                 #  tokenizing word extracted from test.txt file

for y in range(len(tc)):                                                                #
  tempor=model.predict(tc[y])                                                           #  Predicting probability of a language for each word
  dictio.append(tempor)                                                                 #

for i in range (3):                                                                     #
  zer=np.sum(a=dictio,axis=0)                                                           #  Summing individual probabilities

t=np.argmax(zer[0])                                                                     #

if t==0:                                                                                #
  print("English")                                                                      #
if t==1 :                                                                               #
  print("German")                                                                       #  reporting the language 
if t==2 :                                                                               #  with highest cumulative probabilty
  print("French")                                                                       #  to be the dominant one 

['ich', 'heie', 'wasif', 'ich', 'bin', 'ein', 'sehr', 'netter', 'kerl', 'und', 'liebe', 'es', 'neue', 'leute', 'kennenzulernen', 'ich', 'bin', 'sehr', 'begeistert', 'vom', 'maschinellen', 'lernen', 'und', 'strebe', 'an', 'zuknftig', 'forscher', 'zu', 'werden']
German
